In [9]:
import pandas as pd
import numpy as np
from google_play_scraper import app
import plotly.express as px
import datetime as dt

In [2]:
%%time
#US Market
from google_play_scraper import Sort, reviews_all

us_reviews = reviews_all(
    'free.programming.programming',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

CPU times: user 268 ms, sys: 63.6 ms, total: 331 ms
Wall time: 4.36 s


In [3]:
df_new = pd.json_normalize(us_reviews)

In [4]:
df_new.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,c0573d36-689a-4f68-9dc5-507a809488e1,Satyam Dahiwal,https://play-lh.googleusercontent.com/a-/AOh14...,"Since new update, cannot watch the course vide...",5,2,10.0.1.05,2022-06-10 14:39:08,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:05
1,613c3f4d-a6d7-40e6-9f45-ac35c07c7bd5,jatin singh,https://play-lh.googleusercontent.com/a-/AOh14...,App crashes as soon as I open the video lectur...,5,0,10.0.1.05,2022-06-10 11:08:18,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:35:22
2,b95d1be7-bdca-44ea-bfea-96efbda3e6d9,James Zenos,https://play-lh.googleusercontent.com/a-/AOh14...,Excellent App,5,0,10.0.1.05,2022-06-10 10:57:53,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:36:13
3,2fa48bba-12be-4546-b980-5d819c338261,Rohit Thakur,https://play-lh.googleusercontent.com/a/AATXAJ...,Best app for making us ready for the coding job.,3,0,10.0.1.05,2022-06-09 12:13:11,None,NaT
4,9d3945c2-fda6-4807-8a04-ed8bad4c41d5,Narasimha Raju,https://play-lh.googleusercontent.com/a/AATXAJ...,Why the vedios in the app are not opening can ...,1,0,10.0.1.05,2022-06-09 10:02:04,"Hi, sorry for the inconvenience caused. This p...",2022-06-10 10:34:59


In [5]:
df_new.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt'],
      dtype='object')

In [11]:
df_new['date'] = df_new['at'].dt.strftime('%Y-%m-%d')
df_new['time'] = df_new['at'].dt.strftime('%H:%M:%S')

In [13]:
df = df_new[['reviewId','date','time','reviewCreatedVersion','score','thumbsUpCount','content','repliedAt','replyContent']]

In [19]:
df['content'] = df['content'].astype('str')

/tmp/ipykernel_62129/786119033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].astype('str')


In [14]:
df.head()

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,repliedAt,replyContent
0,c0573d36-689a-4f68-9dc5-507a809488e1,2022-06-10,14:39:08,10.0.1.05,5,2,"Since new update, cannot watch the course vide...",2022-06-10 10:36:05,"Hi, sorry for the inconvenience caused. This p..."
1,613c3f4d-a6d7-40e6-9f45-ac35c07c7bd5,2022-06-10,11:08:18,10.0.1.05,5,0,App crashes as soon as I open the video lectur...,2022-06-10 10:35:22,"Hi, sorry for the inconvenience caused. This p..."
2,b95d1be7-bdca-44ea-bfea-96efbda3e6d9,2022-06-10,10:57:53,10.0.1.05,5,0,Excellent App,2022-06-10 10:36:13,"Hi, sorry for the inconvenience caused. This p..."
3,2fa48bba-12be-4546-b980-5d819c338261,2022-06-09,12:13:11,10.0.1.05,3,0,Best app for making us ready for the coding job.,NaT,None
4,9d3945c2-fda6-4807-8a04-ed8bad4c41d5,2022-06-09,10:02:04,10.0.1.05,1,0,Why the vedios in the app are not opening can ...,2022-06-10 10:34:59,"Hi, sorry for the inconvenience caused. This p..."


In [17]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [20]:
df['result']=df['content'].apply(lambda x: sentiment_analysis(x))

/tmp/ipykernel_62129/1969411930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result']=df['content'].apply(lambda x: sentiment_analysis(x))


In [21]:
df['sentiment']=df['result'].apply(lambda x: (x[0]['label']))

df['sentiment_score']=df['result'].apply(lambda x: x[0]['score'])

/tmp/ipykernel_62129/692644880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment']=df['result'].apply(lambda x: (x[0]['label']))


In [22]:
df.head()

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,repliedAt,replyContent,result,sentiment,sentiment_score
0,c0573d36-689a-4f68-9dc5-507a809488e1,2022-06-10,14:39:08,10.0.1.05,5,2,"Since new update, cannot watch the course vide...",2022-06-10 10:36:05,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.991502404212...",NEGATIVE,0.991502
1,613c3f4d-a6d7-40e6-9f45-ac35c07c7bd5,2022-06-10,11:08:18,10.0.1.05,5,0,App crashes as soon as I open the video lectur...,2022-06-10 10:35:22,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.995246946811...",NEGATIVE,0.995247
2,b95d1be7-bdca-44ea-bfea-96efbda3e6d9,2022-06-10,10:57:53,10.0.1.05,5,0,Excellent App,2022-06-10 10:36:13,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'POSITIVE', 'score': 0.998693287372...",POSITIVE,0.998693
3,2fa48bba-12be-4546-b980-5d819c338261,2022-06-09,12:13:11,10.0.1.05,3,0,Best app for making us ready for the coding job.,NaT,None,"[{'label': 'POSITIVE', 'score': 0.998822152614...",POSITIVE,0.998822
4,9d3945c2-fda6-4807-8a04-ed8bad4c41d5,2022-06-09,10:02:04,10.0.1.05,1,0,Why the vedios in the app are not opening can ...,2022-06-10 10:34:59,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.999441683292...",NEGATIVE,0.999442


In [24]:
df[df['sentiment_score']<0.9]

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,repliedAt,replyContent,result,sentiment,sentiment_score
88,43cc0dbe-e493-4bef-969c-349a194b4709,2021-12-12,04:59:25,10.0.0.08,5,0,Job,NaT,None,"[{'label': 'NEGATIVE', 'score': 0.863539218902...",NEGATIVE,0.863539
198,5c890df2-efe5-4604-994d-a13623e3c9da,2021-07-01,19:59:09,9.2.2.0,5,0,GKG,NaT,None,"[{'label': 'NEGATIVE', 'score': 0.884272515773...",NEGATIVE,0.884273
379,2abb6440-48c4-4e5b-be85-bcb68681ce98,2020-09-11,06:58:30,None,5,1,gh,NaT,None,"[{'label': 'POSITIVE', 'score': 0.724450528621...",POSITIVE,0.724451
541,66dab724-45ab-4cec-8b14-72341da893e3,2020-03-12,15:01:10,9.1.6,4,50,All the contents which we can see on actual si...,NaT,None,"[{'label': 'NEGATIVE', 'score': 0.744360089302...",NEGATIVE,0.744360
651,c761714d-156a-415f-9503-2ba2792ad268,2019-10-29,12:14:22,9.1.5,5,19,Can there be a feature to re-order and priorit...,NaT,None,"[{'label': 'POSITIVE', 'score': 0.845586240291...",POSITIVE,0.845586
744,b81adb09-6d65-47db-a1aa-db4afd6ca1a0,2019-08-08,22:57:19,9.1.4,5,1,badhiya hi ar kya,NaT,None,"[{'label': 'NEGATIVE', 'score': 0.633747935295...",NEGATIVE,0.633748
839,e1b23efb-6c95-4e4e-9bc8-43d7e2041a13,2019-06-05,19:06:12,None,4,2,I can pay to remove ads. I know many others ca...,NaT,None,"[{'label': 'POSITIVE', 'score': 0.824978291988...",POSITIVE,0.824978
928,f9f11e03-b569-4c4e-92d4-c6b15a5332c3,2018-12-30,20:27:05,9.0.11,5,1,Primer LLC,NaT,None,"[{'label': 'NEGATIVE', 'score': 0.811849355697...",NEGATIVE,0.811849
1144,61c2c373-5e86-46a1-bd33-0c0c960b87f8,2018-05-03,21:20:17,None,5,1,AAA,NaT,None,"[{'label': 'POSITIVE', 'score': 0.895505547523...",POSITIVE,0.895506
1156,a497060c-02eb-45c1-8795-2363ea5d8421,2018-04-17,08:04:32,9.0.8,4,7,"Previous version was better.Also,where is geek...",NaT,None,"[{'label': 'NEGATIVE', 'score': 0.883154690265...",NEGATIVE,0.883155


In [29]:
df['content_length'] = df['content'].apply(lambda x: len(x.split(' ')))

In [30]:
df.head()

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,repliedAt,replyContent,result,sentiment,sentiment_score,content_length
0,c0573d36-689a-4f68-9dc5-507a809488e1,2022-06-10,14:39:08,10.0.1.05,5,2,"Since new update, cannot watch the course vide...",2022-06-10 10:36:05,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.991502404212...",NEGATIVE,0.991502,26
1,613c3f4d-a6d7-40e6-9f45-ac35c07c7bd5,2022-06-10,11:08:18,10.0.1.05,5,0,App crashes as soon as I open the video lectur...,2022-06-10 10:35:22,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.995246946811...",NEGATIVE,0.995247,26
2,b95d1be7-bdca-44ea-bfea-96efbda3e6d9,2022-06-10,10:57:53,10.0.1.05,5,0,Excellent App,2022-06-10 10:36:13,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'POSITIVE', 'score': 0.998693287372...",POSITIVE,0.998693,2
3,2fa48bba-12be-4546-b980-5d819c338261,2022-06-09,12:13:11,10.0.1.05,3,0,Best app for making us ready for the coding job.,NaT,None,"[{'label': 'POSITIVE', 'score': 0.998822152614...",POSITIVE,0.998822,10
4,9d3945c2-fda6-4807-8a04-ed8bad4c41d5,2022-06-09,10:02:04,10.0.1.05,1,0,Why the vedios in the app are not opening can ...,2022-06-10 10:34:59,"Hi, sorry for the inconvenience caused. This p...","[{'label': 'NEGATIVE', 'score': 0.999441683292...",NEGATIVE,0.999442,14


In [32]:
df.columns

Index(['reviewId', 'date', 'time', 'reviewCreatedVersion', 'score',
       'thumbsUpCount', 'content', 'repliedAt', 'replyContent', 'result',
       'sentiment', 'sentiment_score', 'content_length'],
      dtype='object')

In [33]:
df_final = df[['reviewId', 'date', 'time', 'reviewCreatedVersion', 'score',
       'thumbsUpCount', 'content','sentiment', 'sentiment_score', 'content_length','repliedAt', 'replyContent']]

In [34]:
df_final.head()

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,sentiment,sentiment_score,content_length,repliedAt,replyContent
0,c0573d36-689a-4f68-9dc5-507a809488e1,2022-06-10,14:39:08,10.0.1.05,5,2,"Since new update, cannot watch the course vide...",NEGATIVE,0.991502,26,2022-06-10 10:36:05,"Hi, sorry for the inconvenience caused. This p..."
1,613c3f4d-a6d7-40e6-9f45-ac35c07c7bd5,2022-06-10,11:08:18,10.0.1.05,5,0,App crashes as soon as I open the video lectur...,NEGATIVE,0.995247,26,2022-06-10 10:35:22,"Hi, sorry for the inconvenience caused. This p..."
2,b95d1be7-bdca-44ea-bfea-96efbda3e6d9,2022-06-10,10:57:53,10.0.1.05,5,0,Excellent App,POSITIVE,0.998693,2,2022-06-10 10:36:13,"Hi, sorry for the inconvenience caused. This p..."
3,2fa48bba-12be-4546-b980-5d819c338261,2022-06-09,12:13:11,10.0.1.05,3,0,Best app for making us ready for the coding job.,POSITIVE,0.998822,10,NaT,None
4,9d3945c2-fda6-4807-8a04-ed8bad4c41d5,2022-06-09,10:02:04,10.0.1.05,1,0,Why the vedios in the app are not opening can ...,NEGATIVE,0.999442,14,2022-06-10 10:34:59,"Hi, sorry for the inconvenience caused. This p..."


In [36]:
df_final[df_final['content_length']==df_final['content_length'].max()]

,reviewId,date,time,reviewCreatedVersion,score,thumbsUpCount,content,sentiment,sentiment_score,content_length,repliedAt,replyContent
352,bf805781-bf3b-4430-bf5b-067567f12f3d,2020-10-24,01:04:12,9.1.6,5,6,I usually read from this app when i cannot use...,POSITIVE,0.998935,134,NaT,None


In [37]:
df_final.to_csv('/home/gagand-ubuntu/Projects/sentiment_analysis/sent.csv')